In [1]:
import torch
import torchvision
import torchvision.transforms as transforms
import torch.optim as optim
import torch.nn as nn
import sys
sys.path.append('../models')
from vgg import VGG16_model

In [2]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

cpu


### Load the Pocovid Dataset

In [3]:
import sys
sys.path.append('../scripts')
from pocovid_dataset import PocovidDataset
from torch.utils.data import Dataset, DataLoader

In [4]:
train_pct = 0.8
batch_size = 4
num_workers = 2

dataset = PocovidDataset(root_dir='../data/image_dataset',
                                     transform=transforms.Compose([
                                       transforms.Resize((224,224)),
                                       transforms.RandomAffine(10,translate=(0.1,0.1)),
                                       transforms.ToTensor()
                                       ]))

dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True, num_workers=num_workers)

train_len = int(len(dataset) * train_pct)

trainset, valset = torch.utils.data.random_split(dataset, [train_len, len(dataset) - train_len])

train_loader = torch.utils.data.DataLoader(trainset, num_workers=num_workers, shuffle=True,
                                  batch_size=batch_size, drop_last=True)

val_loader = torch.utils.data.DataLoader(valset, num_workers=num_workers, shuffle=False,
                                batch_size=batch_size)

In [5]:
class_map = {'covid' : dataset.get_covid_class_idx(),
            'pneumonia' : dataset.get_pneu_class_idx(),
            'regular' : dataset.get_regular_class_idx()}

### Model

In [6]:
model = VGG16_model().to(device)

### Loss function and Optimizer

In [8]:
criterion = nn.CrossEntropyLoss().to(device)
optimizer = optim.Adam(params = model.parameters(), lr=1e-4)

### Training Loop

In [9]:
for epoch in range(2):  # loop over the dataset multiple times

    running_loss = 0.0
    for i, data in enumerate(train_loader, 0):
        print(i, data[0].shape)
        # get the inputs; data is a list of [inputs, labels]
        inputs, labels = data[0].to(device), data[1].to(device)

        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = model(inputs)
        print(outputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        # print statistics
        running_loss += loss.item()
#         if i % 2000 == 1999:    # print every 2000 mini-batches
        print('[%d, %5d] loss: %.3f' %
              (epoch + 1, i + 1, running_loss / 2000))
        running_loss = 0.0

print('Finished Training')

0 torch.Size([4, 3, 224, 224])
tensor([[-0.9078,  0.0598, -0.7287],
        [-0.7274,  0.1151, -0.2628],
        [ 0.0155,  0.0108, -0.0131],
        [ 0.5456, -1.0515,  0.9707]], device='cuda:0', grad_fn=<AddmmBackward>)
[1,     1] loss: 0.001
1 torch.Size([4, 3, 224, 224])
tensor([[-1.3089, -0.1077,  0.3587],
        [-0.5954,  0.9401,  0.0754],
        [ 0.1854, -0.5624,  0.0320],
        [-0.1589,  0.6624, -0.3439]], device='cuda:0', grad_fn=<AddmmBackward>)
[1,     2] loss: 0.001
2 torch.Size([4, 3, 224, 224])
tensor([[ 0.3302,  0.1719, -0.1637],
        [-0.5369,  0.2767, -0.4287],
        [-0.6675,  0.2295,  0.6354],
        [-0.2700,  0.2670,  0.8202]], device='cuda:0', grad_fn=<AddmmBackward>)
[1,     3] loss: 0.001
3 torch.Size([4, 3, 224, 224])
tensor([[ 0.6376, -0.6888,  0.3610],
        [ 0.0418, -0.3361, -0.9228],
        [-0.7168,  0.5401,  0.8687],
        [-0.8004,  0.4633, -0.3227]], device='cuda:0', grad_fn=<AddmmBackward>)
[1,     4] loss: 0.001
4 torch.Size([4, 3, 

tensor([[ 0.1160, -0.9463, -0.5324],
        [-0.4256,  0.0664,  1.1214],
        [ 0.4025, -0.4367, -0.2697],
        [-1.4066,  0.2240,  0.3514]], device='cuda:0', grad_fn=<AddmmBackward>)
[1,    40] loss: 0.000
40 torch.Size([4, 3, 224, 224])
tensor([[-0.6295,  0.5489,  0.0188],
        [-0.7005, -0.8606,  1.7665],
        [-0.8628,  1.1046, -0.3213],
        [ 0.0589,  1.4512,  0.2232]], device='cuda:0', grad_fn=<AddmmBackward>)
[1,    41] loss: 0.001
41 torch.Size([4, 3, 224, 224])
tensor([[-0.8216, -0.0708,  0.9758],
        [-0.3098, -0.0071, -0.2833],
        [-0.7519,  0.1347,  0.0800],
        [-0.0314, -0.5195, -0.6541]], device='cuda:0', grad_fn=<AddmmBackward>)
[1,    42] loss: 0.001
42 torch.Size([4, 3, 224, 224])
tensor([[-0.1610,  0.1160,  0.4908],
        [-0.2146,  0.1096, -0.1131],
        [-0.5577,  0.6318, -0.3113],
        [-0.3893, -0.1286, -0.0705]], device='cuda:0', grad_fn=<AddmmBackward>)
[1,    43] loss: 0.001
43 torch.Size([4, 3, 224, 224])
tensor([[-0.5100

[1,    73] loss: 0.000
73 torch.Size([4, 3, 224, 224])
tensor([[-0.3102, -0.0861,  0.1006],
        [-0.2552, -0.1288, -0.0371],
        [-1.0234, -0.1767,  0.1302],
        [-0.8635,  0.1428,  0.1296]], device='cuda:0', grad_fn=<AddmmBackward>)
[1,    74] loss: 0.001
74 torch.Size([4, 3, 224, 224])
tensor([[ 0.4516, -0.3111, -0.4083],
        [-0.8689,  0.2523,  1.0296],
        [-0.8333, -0.1620, -0.0299],
        [ 0.1728,  0.6117, -0.4049]], device='cuda:0', grad_fn=<AddmmBackward>)
[1,    75] loss: 0.000
75 torch.Size([4, 3, 224, 224])
tensor([[ 0.3133, -0.4887, -0.9815],
        [-0.3408,  1.0503,  0.1229],
        [-0.4979, -0.8441,  1.2999],
        [-0.2850,  0.6672, -0.4583]], device='cuda:0', grad_fn=<AddmmBackward>)
[1,    76] loss: 0.000
76 torch.Size([4, 3, 224, 224])
tensor([[ 0.0720, -0.1414, -0.2710],
        [-0.8013,  0.9348,  0.0403],
        [-0.3498, -0.7364,  0.3524],
        [ 0.0862, -0.8028, -0.1886]], device='cuda:0', grad_fn=<AddmmBackward>)
[1,    77] loss:

tensor([[-1.6225,  0.2703,  0.8731],
        [-0.5890,  0.4683, -0.1700],
        [-0.4205,  0.9254, -1.3176],
        [ 0.3892, -0.6339,  0.0915]], device='cuda:0', grad_fn=<AddmmBackward>)
[1,   107] loss: 0.000
107 torch.Size([4, 3, 224, 224])
tensor([[-0.0943, -0.9119,  0.7390],
        [-0.1217,  0.2414, -0.3199],
        [-0.3362,  0.4347, -0.3163],
        [-0.3823, -0.0667, -1.1385]], device='cuda:0', grad_fn=<AddmmBackward>)
[1,   108] loss: 0.001
108 torch.Size([4, 3, 224, 224])
tensor([[-0.2306, -0.0628,  0.3036],
        [-0.8646,  0.3254, -1.1466],
        [-0.8217, -0.0142,  0.0905],
        [-0.6807, -0.1045,  0.0428]], device='cuda:0', grad_fn=<AddmmBackward>)
[1,   109] loss: 0.001
109 torch.Size([4, 3, 224, 224])
tensor([[-1.6541,  1.1145,  0.0103],
        [ 0.5318, -0.5603,  0.7144],
        [-0.5901,  0.3882, -0.5783],
        [-0.8466,  0.3482,  0.3802]], device='cuda:0', grad_fn=<AddmmBackward>)
[1,   110] loss: 0.001
110 torch.Size([4, 3, 224, 224])
tensor([[ 0.

tensor([[-0.6472,  0.8142,  1.0662],
        [-1.4415, -0.6658,  1.0981],
        [-0.0624, -0.1809, -0.7037],
        [ 0.4085,  0.4502, -0.4264]], device='cuda:0', grad_fn=<AddmmBackward>)
[1,   148] loss: 0.000
148 torch.Size([4, 3, 224, 224])
tensor([[-0.1105, -0.2368, -1.1878],
        [-0.9203, -0.1986,  0.0209],
        [ 0.5405, -0.5656,  0.0501],
        [-0.6630, -1.3767,  1.4974]], device='cuda:0', grad_fn=<AddmmBackward>)
[1,   149] loss: 0.000
149 torch.Size([4, 3, 224, 224])
tensor([[-1.1370,  0.5769,  0.4266],
        [-1.1476, -0.0452, -0.0860],
        [ 1.1033, -0.7474, -1.0940],
        [ 0.0843, -0.8367, -0.0871]], device='cuda:0', grad_fn=<AddmmBackward>)
[1,   150] loss: 0.000
150 torch.Size([4, 3, 224, 224])
tensor([[-0.0489, -0.2335, -0.0258],
        [-0.7471,  0.6590, -0.2330],
        [-1.4342, -0.9243,  1.3828],
        [ 1.0373,  0.3121, -1.6123]], device='cuda:0', grad_fn=<AddmmBackward>)
[1,   151] loss: 0.000
151 torch.Size([4, 3, 224, 224])
tensor([[-0.

8 torch.Size([4, 3, 224, 224])
tensor([[ 0.5031, -0.4179, -0.6554],
        [-0.9076, -0.6144,  0.7816],
        [-0.8930, -0.2216,  0.0427],
        [-0.5159, -0.2354,  0.0825]], device='cuda:0', grad_fn=<AddmmBackward>)
[2,     9] loss: 0.000
9 torch.Size([4, 3, 224, 224])
tensor([[ 0.9829, -0.0829, -0.8418],
        [ 0.6313, -0.5490,  0.4448],
        [ 0.2437, -0.5453, -0.4735],
        [-2.0164,  0.2598,  1.0100]], device='cuda:0', grad_fn=<AddmmBackward>)
[2,    10] loss: 0.000
10 torch.Size([4, 3, 224, 224])
tensor([[-0.5162,  0.6224, -0.9111],
        [ 0.0517, -1.2378,  1.7035],
        [ 0.8305,  0.1020, -1.1350],
        [ 0.2170,  0.2192,  0.8883]], device='cuda:0', grad_fn=<AddmmBackward>)
[2,    11] loss: 0.000
11 torch.Size([4, 3, 224, 224])
tensor([[-0.7319,  0.0251,  0.4917],
        [ 0.4718,  0.0268, -0.7705],
        [-0.0816,  0.4598, -0.4477],
        [ 0.4019, -0.0411,  0.5769]], device='cuda:0', grad_fn=<AddmmBackward>)
[2,    12] loss: 0.001
12 torch.Size([4, 

tensor([[-0.5036,  0.2929, -0.3035],
        [ 0.7360,  0.2389,  0.4346],
        [-1.6014,  0.1527,  1.3609],
        [ 0.6755,  0.2644, -1.3759]], device='cuda:0', grad_fn=<AddmmBackward>)
[2,    50] loss: 0.000
50 torch.Size([4, 3, 224, 224])
tensor([[-1.1714,  0.2451,  1.2305],
        [ 1.5423, -0.7506, -0.7991],
        [-1.4645,  1.1806, -0.3718],
        [ 0.1863, -0.6578,  0.7866]], device='cuda:0', grad_fn=<AddmmBackward>)
[2,    51] loss: 0.000
51 torch.Size([4, 3, 224, 224])
tensor([[ 0.8394, -0.1134, -1.1347],
        [ 0.8306, -0.8484, -0.1506],
        [-1.2053, -0.0780,  0.6738],
        [-1.3708,  0.2838,  0.6295]], device='cuda:0', grad_fn=<AddmmBackward>)
[2,    52] loss: 0.000
52 torch.Size([4, 3, 224, 224])
tensor([[ 0.8859, -1.7595,  0.5024],
        [-0.1763, -0.1414, -0.7485],
        [-1.8896,  0.9315,  2.4539],
        [-0.1577,  0.5588, -0.9135]], device='cuda:0', grad_fn=<AddmmBackward>)
[2,    53] loss: 0.000
53 torch.Size([4, 3, 224, 224])
tensor([[ 1.1320

tensor([[-0.5652, -0.0776, -0.8164],
        [ 0.4069,  0.7185,  0.1377],
        [-0.0978, -0.2042,  0.8170],
        [-0.2123, -0.9045, -0.6256]], device='cuda:0', grad_fn=<AddmmBackward>)
[2,    90] loss: 0.000
90 torch.Size([4, 3, 224, 224])
tensor([[ 0.4525, -0.2594, -2.4396],
        [-1.1796,  0.6154,  0.3026],
        [-0.0535,  0.7348,  0.2918],
        [-1.2473, -0.7693,  1.0457]], device='cuda:0', grad_fn=<AddmmBackward>)
[2,    91] loss: 0.000
91 torch.Size([4, 3, 224, 224])
tensor([[ 0.2351, -0.4017, -1.7927],
        [-1.0837, -1.0555,  2.2788],
        [ 0.0357,  0.0319,  0.2470],
        [ 0.2746, -0.3578, -0.5922]], device='cuda:0', grad_fn=<AddmmBackward>)
[2,    92] loss: 0.000
92 torch.Size([4, 3, 224, 224])
tensor([[-0.8524, -0.6516,  0.1563],
        [ 0.2470, -0.0375,  0.1584],
        [ 0.3284, -0.8220,  0.1011],
        [-1.9884,  1.5246, -0.5890]], device='cuda:0', grad_fn=<AddmmBackward>)
[2,    93] loss: 0.000
93 torch.Size([4, 3, 224, 224])
tensor([[ 0.0811

tensor([[-0.8166, -0.2155,  1.1998],
        [ 0.6060, -0.9108,  0.3236],
        [-0.3206,  0.6979, -0.9467],
        [ 0.3006, -0.1745,  0.0052]], device='cuda:0', grad_fn=<AddmmBackward>)
[2,   130] loss: 0.001
130 torch.Size([4, 3, 224, 224])
tensor([[-1.2737, -0.2769,  0.0089],
        [-1.4263,  0.4329,  1.0713],
        [ 0.5195, -0.4097, -1.1205],
        [ 1.9213, -1.5039, -0.0804]], device='cuda:0', grad_fn=<AddmmBackward>)
[2,   131] loss: 0.001
131 torch.Size([4, 3, 224, 224])
tensor([[-1.9369,  0.7601,  0.9278],
        [-0.1822, -0.1814, -0.7500],
        [ 2.3144, -1.4575, -0.5385],
        [ 0.0733,  0.5455,  0.3873]], device='cuda:0', grad_fn=<AddmmBackward>)
[2,   132] loss: 0.000
132 torch.Size([4, 3, 224, 224])
tensor([[ 0.0456, -0.4998,  0.8855],
        [ 1.3316, -1.3300,  0.0494],
        [-0.4152,  0.7993, -0.1118],
        [-0.1340,  0.0812, -0.1961]], device='cuda:0', grad_fn=<AddmmBackward>)
[2,   133] loss: 0.000
133 torch.Size([4, 3, 224, 224])
tensor([[ 0.

tensor([[-3.1195e-02,  3.2822e-02,  4.9305e-01],
        [-5.6187e-03, -4.9055e-01, -5.7959e-01],
        [-5.0375e-01, -6.0568e-01, -1.1304e+00],
        [ 7.0396e-04, -1.7235e+00,  5.8749e-03]], device='cuda:0',
       grad_fn=<AddmmBackward>)
[2,   170] loss: 0.000
170 torch.Size([4, 3, 224, 224])
tensor([[-0.1550, -0.1110,  0.2622],
        [ 1.8542, -1.0479,  0.4132],
        [-0.0307,  0.1294,  0.5739],
        [-0.6975,  0.1180, -0.3891]], device='cuda:0', grad_fn=<AddmmBackward>)
[2,   171] loss: 0.000
171 torch.Size([4, 3, 224, 224])
tensor([[-0.2278, -0.1910,  0.2323],
        [-0.3919, -0.6554, -0.1984],
        [-0.4398,  0.0477,  0.5576],
        [ 0.0302, -1.1324, -1.6927]], device='cuda:0', grad_fn=<AddmmBackward>)
[2,   172] loss: 0.000
172 torch.Size([4, 3, 224, 224])
tensor([[-0.9160, -0.8931,  2.1980],
        [-0.4091,  0.5486, -1.9723],
        [-0.4735,  0.3728,  0.6603],
        [ 0.2794, -0.4103, -1.2738]], device='cuda:0', grad_fn=<AddmmBackward>)
[2,   173] lo